In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
import seaborn as sns

import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from batchbald_redux import repeated_mnist, active_learning, batchbald
from main.models import ConvNet
from main.training_models import test_performance
from main.utils import save_experiment, load_experiment, log_experiment, generate_experiment_id
from laplace.curvature import AsdlGGN, AsdlGGN
from main.laplace_batch import get_laplace_batch
from dataclasses import dataclass
from main.active_learning import run_active_learning

sns.set_palette(sns.color_palette("Spectral"))

%reload_ext autoreload
%autoreload 2

c:\Users\vince\Documents\Statistics\TT\msc_thesis\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Settings for Active Learning

In [3]:
# set configurations
@dataclass
class ActiveLearningConfig:
    subset_of_weights: str = 'last_layer'
    hessian_structure: str = 'kron'
    backend: str = 'AsdlGGN'
    temperature: float = 1
    max_training_samples: int = 100
    acquisition_batch_size: int = 5
    al_method: str = 'entropy'
    test_batch_size: int = 512
    num_classes: int = 10
    num_initial_samples: int = 40
    training_iterations: int = 4096 * 6
    scoring_batch_size: int = 64
    train_batch_size: int = 64
    extract_pool: int = 55000  # number of samples to extract from the dataset (bit of a hack)

experiment_name = 'lowtemperature_'  # provide descriptive name for the experiment
experiment_name += generate_experiment_id()

config = ActiveLearningConfig()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
kwargs = {"num_workers": 1, "pin_memory": True}

save_results = False

Using device: cpu


## Load and Prepare Data

In [4]:
# loading data
train_dataset, test_dataset = repeated_mnist.create_MNIST_dataset()

# get indices of initial samples
initial_samples = active_learning.get_balanced_sample_indices(
    repeated_mnist.get_targets(train_dataset), num_classes=config.num_classes, n_per_digit=config.num_initial_samples / config.num_classes
)

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=config.test_batch_size, shuffle=False, **kwargs)

active_learning_data = active_learning.ActiveLearningData(train_dataset)

# Split off the initial samples first.
active_learning_data.acquire(initial_samples)

# THIS REMOVES MOST OF THE POOL DATA. REMOVE THIS LINE TO USE THE FULL POOL
active_learning_data.extract_dataset_from_pool(config.extract_pool)

train_loader = torch.utils.data.DataLoader(
    active_learning_data.training_dataset,
    sampler=active_learning.RandomFixedLengthSampler(active_learning_data.training_dataset, config.training_iterations),
    batch_size=config.train_batch_size,
    **kwargs,
)

pool_loader = torch.utils.data.DataLoader(
    active_learning_data.pool_dataset, batch_size=config.scoring_batch_size, shuffle=False, **kwargs
)

c:\Users\vince\Documents\Statistics\TT\msc_thesis\.venv\Lib\site-packages\torch\utils\data\sampler.py:64: UserWarning: `data_source` argument is not used and will be removed in 2.2.0.You may still have custom implementation that utilizes it.
  warnings.warn("`data_source` argument is not used and will be removed in 2.2.0."


## Runs Active Learning with settings

In [4]:
results = run_active_learning(
    train_loader=train_loader,
    test_loader=test_loader, 
    pool_loader=pool_loader,
    active_learning_data=active_learning_data,
    model_constructor=ConvNet, 
    config=config, 
    device=device
    )

Training Set Size:  40%|████      | 40/100 [00:00<?, ?it/s]2024-07-28 11:14:36,897 - INFO - Training set size: 40, Test set accuracy: 76.53, Test set loss: -0.0117
c:\Users\vince\Documents\Statistics\TT\msc_thesis\.venv\Lib\site-packages\torch\nn\modules\module.py:1373: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
c:\Users\vince\Documents\Statistics\TT\msc_thesis\.venv\Lib\site-packages\laplace\baselaplace.py:409: UserWarning: By default `link_approx` is `probit`. Make sure to set it equals to the way you want to call `la(test_data, pred_type=..., link_approx=...)`.
  warnings.warn(
Computing entropies: 78it [00:04, 17.98it/s]
Training Set Size:  45%|████▌     | 45/100 [00:3

Dataset indices:  [26852  8713 13063  1804 56874]
Scores:  [1.9689174890518188, 1.9535801410675049, 1.9389352798461914, 1.9305953979492188, 1.9218602180480957]
Labels:  tensor([8, 5, 9, 8, 8])


2024-07-28 11:15:15,546 - INFO - Training set size: 45, Test set accuracy: 77.00, Test set loss: -0.0225
Computing entropies: 78it [00:04, 15.73it/s]
Training Set Size:  50%|█████     | 50/100 [01:15<06:22,  7.64s/it]

Dataset indices:  [48680 12076 12359 32012  8786]
Scores:  [1.9359958171844482, 1.6774601936340332, 1.6574766635894775, 1.6573704481124878, 1.6566517353057861]
Labels:  tensor([5, 9, 6, 8, 3])


2024-07-28 11:15:53,676 - INFO - Training set size: 50, Test set accuracy: 79.31, Test set loss: -0.0166
Computing entropies: 78it [00:04, 16.19it/s]
Training Set Size:  55%|█████▌    | 55/100 [01:52<05:37,  7.50s/it]

...

2024-07-28 11:20:42,566 - INFO - Training set size: 90, Test set accuracy: 83.68, Test set loss: -0.0266
Computing entropies: 77it [00:04, 17.16it/s]
Training Set Size:  95%|█████████▌| 95/100 [06:40<00:36,  7.30s/it]

Dataset indices:  [13997 14355 36760 11536 41283]
Scores:  [1.7375097274780273, 1.7242283821105957, 1.7170239686965942, 1.690377116203308, 1.6881840229034424]
Labels:  tensor([9, 2, 7, 9, 3])


2024-07-28 11:21:18,216 - INFO - Training set size: 95, Test set accuracy: 83.04, Test set loss: -0.0249
Computing entropies: 77it [00:04, 16.78it/s]
Training Set Size: 100%|██████████| 100/100 [07:17<00:00,  7.30s/it]

Dataset indices:  [38631  9860 24587  1724 14124]
Scores:  [1.6712132692337036, 1.629883885383606, 1.543965458869934, 1.4847278594970703, 1.471351981163025]
Labels:  tensor([4, 6, 8, 2, 2])


2024-07-28 11:21:56,252 - INFO - Training set size: 100, Test set accuracy: 79.66, Test set loss: -0.0177
Training Set Size: 100%|██████████| 100/100 [07:34<00:00,  7.57s/it]


## Analyse Results

In [5]:
sns.lineplot(x=np.arange(start=config.num_initial_samples, stop=config.max_training_samples + 1, step=config.acquisition_batch_size),
            y=results['test_accs'])
plt.title('Active Learning Performance')
plt.xlabel('Number of training samples')
plt.ylabel('Test Accuracy')

NameError: name 'results' is not defined

## Save Experiment Results

In [7]:
if True:
    save_experiment(config, results, experiment_name)

Experiment saved in experiments\temperature_20240728-111421
